In [102]:
import pandas as pd
import numpy as np

In [103]:
df_lumiere = pd.read_csv('/home/johannes/Data/SSD_2.0TB/GNN_pCR/data/brain_cancer/Lumiere/LUMIERE-ExpertRating-v202211.csv')
df_lumiere = df_lumiere[["Patient", "Date", "Rating (according to RANO, PD: Progressive disease, SD: Stable disease, PR: Partial response, CR: Complete response, Pre-Op: Pre-Operative, Post-Op: Post-Operative)"]]
df_lumiere["Rating (according to RANO, PD: Progressive disease, SD: Stable disease, PR: Partial response, CR: Complete response, Pre-Op: Pre-Operative, Post-Op: Post-Operative)"] = df_lumiere["Rating (according to RANO, PD: Progressive disease, SD: Stable disease, PR: Partial response, CR: Complete response, Pre-Op: Pre-Operative, Post-Op: Post-Operative)"].map({
    "PD": "Progressive disease",
    "SD": "Stable disease",
    "PR": "Partial response",
    "CR": "Complete response",
    "Pre-Op": "Pre-Operative",
    "Post-Op": "Post-Operative"
})
df_lumiere = df_lumiere.rename(columns={
    "Patient": "Patient",
    "Date": "Date",
    "Rating (according to RANO, PD: Progressive disease, SD: Stable disease, PR: Partial response, CR: Complete response, Pre-Op: Pre-Operative, Post-Op: Post-Operative)": "Response"
})
df_lumiere.head()

,Patient,Date,Response
0,Patient-001,week-000-1,Pre-Operative
1,Patient-001,week-000-2,Post-Operative
2,Patient-001,week-044,Stable disease
3,Patient-001,week-056,Progressive disease
4,Patient-002,week-000,Pre-Operative


In [104]:
df_burdenko = pd.read_csv('/home/johannes/Data/SSD_2.0TB/GNN_pCR/data/brain_cancer/Burdenko/Burdenko GBM clinical data.csv')
df_burdenko = df_burdenko[["AnonymPatientID", "AnonymStudyDate", 'Anonym1st_fup',
                            'Anonym2nd_fup', 'Anonym3rd_fup', 'Anonym4th_fup', 'Anonym5th_fup',
                            'Anonym6th_fup', 'Anonym7th_fup', 'Anonym8th_fup', 'Anonym9th_fup',
                            'Anonym10th_fup', 'Response_1st_fup', 'Response_2nd_fup', 'Response_3rd_fup',
                            'Response_4th_fup', 'Response_5th_fup', 'Response_6th_fup',
                            'Response_7th_fup', 'Response_8th_fup', 'Response_9th_fup',
                            'Response_10th_fup']]
data = ["PlanningCT" for _ in range(df_burdenko.shape[0])]
df_burdenko.insert(loc=12, column="Response_0_fup", value=data)

patient_id_list = []
dates_list = []
responses_list = []

for i in range(df_burdenko.shape[0]):

    try:

        patient_id = [df_burdenko.iloc[i].replace(" ", np.nan).dropna().iloc[0]]
        
        data = df_burdenko.iloc[i].replace(" ", np.nan).dropna().iloc[1:]
        dates = [fup for fup in data.index if "Anonym" in fup]
        dates = data.iloc[:len(dates)].tolist()
        responses = [resp for resp in data.index if "Response" in resp]
        responses = data.iloc[len(dates):len(dates)+len(responses)].tolist()
        length = np.min([len(dates), len(responses)])
        
        patient_id = patient_id * length
        dates = dates[:length]
        responses = responses[:length]

        assert len(patient_id) == len(dates) == len(responses)

        patient_id_list.extend(patient_id)
        dates_list.extend(dates)
        responses_list.extend(responses)
    
    except:
        print(patient_id)
        print(dates)
        print(responses)
        print(f"Skipping row {i} due to error.")

df_burdenko = pd.DataFrame({
    "Patient": patient_id_list,
    "Date": dates_list,
    "Response": responses_list
})

df_burdenko

,Patient,Date,Response
0,Burdenko-GBM-001,5/6/04,PlanningCT
1,Burdenko-GBM-001,7/8/04,response
2,Burdenko-GBM-001,8/16/04,stable
3,Burdenko-GBM-001,10/4/04,stable
4,Burdenko-GBM-001,12/23/04,stable
...,...,...,...
826,Burdenko-GBM-180,11/13/04,response
827,Burdenko-GBM-180,12/9/04,response
828,Burdenko-GBM-180,1/13/05,progression
829,Burdenko-GBM-180,4/23/05,progression


In [110]:
df = pd.concat([df_lumiere, df_burdenko], ignore_index=True)
df["Response"] = df["Response"].replace("Progressive disease", "progression")
df["Response"] = df["Response"].replace("Stable disease", "stable")
df["Response"] = df["Response"].replace("Complete response", "response")

df["Response"] = df["Response"].replace("pseudo progression", "delete")
df["Response"] = df["Response"].replace("Partial response", "delete")
df["Response"] = df["Response"].replace("distant progression", "delete")
df["Response"] = df["Response"].replace("response, distant progression", "delete")
df["Response"] = df["Response"].replace("stable, distant progression", "delete")
df["Response"] = df["Response"].replace("response, pseudo progression", "delete")

delete_patients = []

for idx in df[df["Response"] == "delete"].index.tolist():
    delete_patients.append(df.iloc[idx]["Patient"])

df = df[~df["Patient"].isin(delete_patients)]

df["Response"] = df["Response"].replace("Pre-Operative", np.nan)
df = df.dropna(subset=["Response"])

df.to_csv("./Lumiere_Burdenko_Responses.csv", index=False)
print(df.Response.value_counts())
print(df.Patient.nunique())

Response
progression       329
stable            274
response          161
PlanningCT        135
Post-Operative    107
Name: count, dtype: int64
213
